In [1]:
import re

import numpy as np
import pandas as pd

In [2]:
players = pd.read_csv('../data/data.csv', encoding='utf-8')
league_team = pd.read_csv('../data/league_team_mapping.csv', encoding='utf-8')
league_rev = pd.read_csv('../data/revenue_to_league_mapping.csv', encoding='utf-8')

In [3]:
league = pd.merge(league_team, league_rev, on='League', how='left')
players = pd.merge(players, league, left_on='Club', right_on='Team', how='left')

In [4]:
players

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Team,League,Revenue,Revenue_per_team
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,6.0,11.0,15.0,14.0,8.0,€226.5M,FC Barcelona,La Liga,4479.0,223.95
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,7.0,11.0,15.0,14.0,11.0,€127.1M,Juventus,Serie A,2163.0,108.20
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,9.0,9.0,15.0,15.0,11.0,€228.1M,Paris Saint-Germain,Ligue 1,1692.0,84.60
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,90.0,85.0,87.0,88.0,94.0,€138.6M,Manchester United,Premier League,6562.0,328.10
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,15.0,13.0,5.0,10.0,13.0,€196.4M,Manchester City,Premier League,6562.0,328.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,18202,238813,J. Lundstram,19,https://cdn.sofifa.org/players/4/19/238813.png,England,https://cdn.sofifa.org/flags/14.png,47,65,Crewe Alexandra,...,10.0,13.0,7.0,8.0,9.0,€143K,Crewe Alexandra,EFL League Two,106.0,4.40
18203,18203,243165,N. Christoffersson,19,https://cdn.sofifa.org/players/4/19/243165.png,Sweden,https://cdn.sofifa.org/flags/46.png,47,63,Trelleborgs FF,...,10.0,9.0,9.0,5.0,12.0,€113K,Trelleborgs FF,Allsvenskan,145.0,9.10
18204,18204,241638,B. Worman,16,https://cdn.sofifa.org/players/4/19/241638.png,England,https://cdn.sofifa.org/flags/14.png,47,67,Cambridge United,...,6.0,5.0,10.0,6.0,13.0,€165K,Cambridge United,EFL League Two,106.0,4.40
18205,18205,246268,D. Walker-Rice,17,https://cdn.sofifa.org/players/4/19/246268.png,England,https://cdn.sofifa.org/flags/14.png,47,66,Tranmere Rovers,...,14.0,6.0,14.0,8.0,9.0,€143K,Tranmere Rovers,EFL League Two,106.0,4.40


In [5]:
drop_cols = ['Unnamed: 0', 'ID', 'Name', 'Photo', 'Flag', 'Club Logo', 'Jersey Number', 'Joined', 'Team']
players = players.drop(drop_cols, axis=1, errors='ignore')

In [6]:
def transform_money(df, colnames):
    df_copy = df.copy()
    for col in colnames:
        df_copy[col] = df_copy[col].apply(parse_money)
    return df_copy

def parse_money(x):
    if pd.isnull(x):
        return x
    m = re.search(r'€(\d+\.?\d*)(\w?)', x)
    value = float(m.group(1))
    unit = m.group(2)
    if unit == 'K':
        return value*1000
    if unit == 'M':
        return value*1000000
    return value

In [7]:
def transform_height(df, colnames):
    df_copy = df.copy()
    for col in colnames:
        df_copy[col] = df_copy[col].apply(parse_height)
    return df_copy

def parse_height(x):
    if pd.isnull(x):
        return x
    m = re.search(r'(\d+)\'(\d+)', x)
    return int(m.group(1))*12 + int(m.group(2))

In [8]:
def transform_weight(df, colnames):
    df_copy = df.copy()
    for col in colnames:
        df_copy[col] = df_copy[col].apply(parse_weight)
    return df_copy

def parse_weight(x):
    if pd.isnull(x):
        return x
    m = re.search(r'(\d+)lbs', x)
    return int(m.group(1))

In [9]:
def transform_pos_ratings(df, colnames):
    df_copy = df.copy()
    for col in colnames:
        df_copy[col] = df_copy[col].apply(parse_pos_ratings)
    return df_copy

def parse_pos_ratings(x):
    if pd.isnull(x):
        return x
    m = re.search(r'(\d+)\+\d', x)
    return int(m.group(1))

In [10]:
def transform_loaned(df):
    df_copy = df.copy()
    loaned_players_idx = ~df_copy['Loaned From'].isna()
    df_copy.loc[loaned_players_idx, 'Club'] = df_copy.loc[loaned_players_idx, 'Loaned From']
    df_copy.loc[loaned_players_idx, 'Contract Valid Until'] = df_copy.loc[loaned_players_idx, 'Contract Valid Until'].str.slice(-4)
    df_copy['Loaned Out'] = False
    df_copy.loc[loaned_players_idx, 'Loaned Out'] = True
    df_copy = df_copy.drop('Loaned From', axis=1)
    return df_copy

In [11]:
def transform_work_rate(df):
    df_copy = df.copy()
    df_copy['Work Rate'] = df_copy['Work Rate'].str.split('/')
    df_copy['Off Work Rate'] = df_copy['Work Rate'].str[0]
    df_copy['Def Work Rate'] = df_copy['Work Rate'].str[1]
    df_copy = df_copy.drop('Work Rate', axis=1)
    return df_copy

In [12]:
players = transform_money(players, ['Value', 'Wage', 'Release Clause'])
players = transform_weight(players, ['Weight'])
players = transform_height(players, ['Height'])
players = transform_pos_ratings(players, 'LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB'.split(','))
players = transform_loaned(players)
players = transform_work_rate(players)

In [13]:
players.to_csv('../data/data_cleaned.csv', index=False)